<a href="https://colab.research.google.com/github/Taaniya/semantic-search-for-info-retrieval/blob/main/semantic_search_evaluation_with_ranx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install ranx

In [2]:
import pandas as pd
import json

from ranx.metrics.f1 import _f1_parallel, _f1, f1
from ranx import Qrels, Run, evaluate

In [3]:
qrels_path = "./drive/MyDrive/Colab Notebooks/NLP_exploration/QA_on_docs/qrels_v2.xlsx"
preds_run_path = "./drive/MyDrive/Colab Notebooks/NLP_exploration/QA_on_docs/run.xlsx"

In [4]:
qrels_df = pd.read_excel(qrels_path)
run_df = pd.read_excel(preds_run_path)

In [5]:
qrels_df.head()

,q_id,doc_id,score
0,0,10,5
1,1,10,5
2,2,10,5
3,3,2,5
4,4,2,5


In [6]:
run_df.head()

,q_id,doc_id,cosine_similarity
0,0,10.0,0.341212
1,0,2.0,0.177277
2,1,10.0,0.412571
3,1,3.0,0.168216
4,2,10.0,0.323567


In [7]:
# drop rows where there is no doc_id for a query
run_df = run_df.dropna(subset=['doc_id'])

# cast doc_id to int, then string object
run_df['doc_id'] = run_df['doc_id'].astype('int')

qrels_df['doc_id'] = 'd_' + qrels_df['doc_id'].astype(str)
run_df['doc_id'] = 'd_' + run_df['doc_id'].astype(str)

# cast q_id columns to string type
qrels_df['q_id'] = qrels_df['q_id'].astype(str)
run_df['q_id'] = run_df['q_id'].astype(str)

In [8]:
qrels_df.count()

q_id      18
doc_id    18
score     18
dtype: int64

In [9]:
run_df.count()

q_id                 29
doc_id               29
cosine_similarity    29
dtype: int64

In [10]:
qrels_df.head()

,q_id,doc_id,score
0,0,d_10,5
1,1,d_10,5
2,2,d_10,5
3,3,d_2,5
4,4,d_2,5


In [11]:
run_df.head()

,q_id,doc_id,cosine_similarity
0,0,d_10,0.341212
1,0,d_2,0.177277
2,1,d_10,0.412571
3,1,d_3,0.168216
4,2,d_10,0.323567


In [12]:
# Create qrels object
qrels = Qrels.from_df( df=qrels_df,  q_id_col="q_id", doc_id_col="doc_id")

# Create run object
run = Run.from_df(df=run_df,
                  q_id_col="q_id",
                  doc_id_col="doc_id",
                  score_col="cosine_similarity")

In [13]:
# perform evaluation
score_dict = evaluate(qrels, run, ["mrr", "mrr@1", "mrr@2", "recall@1", "recall@2", "precision@1", "f1@1", "hits@1"])

for k,v in score_dict.items():
  print(f"{k}: {v}")

mrr: 0.9629629629629629
mrr@1: 0.9444444444444444
mrr@2: 0.9444444444444444
recall@1: 0.9444444444444444
recall@2: 0.9444444444444444
precision@1: 0.9444444444444444
f1@1: 0.7777777777777778
hits@1: 0.9444444444444444


In [14]:
# pretty print scores
print(json.dumps(run.scores, indent=4))

{
    "mrr": {
        "0": 1.0,
        "1": 1.0,
        "10": 1.0,
        "11": 1.0,
        "12": 1.0,
        "13": 1.0,
        "14": 1.0,
        "15": 1.0,
        "16": 1.0,
        "17": 1.0,
        "2": 1.0,
        "3": 1.0,
        "4": 1.0,
        "5": 1.0,
        "6": 0.3333333333333333,
        "7": 1.0,
        "8": 1.0,
        "9": 1.0
    },
    "mrr@1": {
        "0": 1.0,
        "1": 1.0,
        "10": 1.0,
        "11": 1.0,
        "12": 1.0,
        "13": 1.0,
        "14": 1.0,
        "15": 1.0,
        "16": 1.0,
        "17": 1.0,
        "2": 1.0,
        "3": 1.0,
        "4": 1.0,
        "5": 1.0,
        "6": 0.0,
        "7": 1.0,
        "8": 1.0,
        "9": 1.0
    },
    "mrr@2": {
        "0": 1.0,
        "1": 1.0,
        "10": 1.0,
        "11": 1.0,
        "12": 1.0,
        "13": 1.0,
        "14": 1.0,
        "15": 1.0,
        "16": 1.0,
        "17": 1.0,
        "2": 1.0,
        "3": 1.0,
        "4": 1.0,
        "5": 1.0,
    

In [15]:
# convert qrels and run object to numba typed list to call functions f1, _f1, _f1_parallel

_qrels = qrels.to_typed_list()
_run = run.to_typed_list()

In [16]:
print(f1(_qrels, _run, 1, 1))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]


In [17]:
print(_f1_parallel(_qrels, _run, 1 , 1))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]


In [18]:
scores = []

for i in range(len(_qrels)):
  try:
      scores.append(_f1(_qrels[i], _run[i], 1 , 1))
  except Exception as error:
    # print division by zero exception
    print(f" {i} An exception occurred:", error)
    scores.append(0)
    continue

 14 An exception occurred: division by zero


In [19]:
scores

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0,
 1.0,
 1.0,
 1.0]